### And now - Week 3 Day 3

## AutoGen Core

Something a little different.

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

### The fundamental principle

Autogen Core decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a Runtime.

There are 2 types: **Standalone** and **Distributed**.

Today we will use a standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.

Tomorrow we'll briefly look at a Distributed runtime.


In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)


True

### First we define our Message object

Whatever structure we want for messages in our Agent framework.

In [2]:
"""
#dataclass means that it won't have any methods, it will just hold data. AG fundamental idea is messaging as compared to 
# State in Langraph. so the thing you start by defining is message
"""

# Let's have a simple one!

@dataclass 
class Message:
    content: str


### Now we define our Agent in AutogenCore

This is different from agent in Autogen Agent Chat. Agent here is just a wrapper, a thing that can be messaged. So agents can talk to other agents. It can created, managed, messaged, referred to. What we do with it is upto us. 

A subclass of RoutedAgent.

Every Agent has an **Agent ID** which has 2 components:  
`agent.id.type` describes the kind of agent it is  
`agent.id.key` gives it its unique identifier

Each agent has a unique type and key that can be used to identify it.

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [ ]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("Simple")

    @message_handler #message decorated with this means it is something that can receive messages of the message class when sent to this type&key id
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagree.") #also returns a message

#there's no llm or ai model, it doesnt matter to autogen core as it only cares about handling of messages
        

### OK let's create a Standalone runtime and register our agent type

In [4]:

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent()) #this says that "simple_agent" is a type of agent that can be spawned/created, so we tell runtime that. We still havent built or instantiated an agent, just informed runtime

AgentType(type='simple_agent')

### Alright! Let's start a runtime and send a message

In [ ]:
runtime.start() #now we start run time

In [6]:
agent_id = AgentId("simple_agent", "default") #this id identifies the agent, id is "default"
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well hi there!' and I disagree.


In [7]:
await runtime.stop()
await runtime.close()

### OK Now let's do something more interesting
We add an llm to this flow now, since adding llm is our job, autogen core just passes messages around.

We'll use an AgentChat Assistant!

In [ ]:

class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client) #this what agent will delegate to when it actually needs some code or llm to run. we could put a random thing here or an assistant agent like here. "_" signifies a private/secret variable

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message: # if the message is not directed to an agent then autogen core will automatically direct the message to handlers based on the type of the message. the later is good for pub/sub agents (publish/subscribe) where multiple agents are e.g. interested in the same topic. this helps scaling and distributing
        print(f"{self.id.type} received message: {message.content}") #say recieved message
        text_message = TextMessage(content=message.content, source="user") #textmessage looks different from message, this is autogent agent chat message
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)
    


In [9]:
from autogen_core import SingleThreadedAgentRuntime

#we register two agents here:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [10]:
"""
Start processing messages in the background. send to llm agent and send its response to simple agent. 
then we send that back again to llm agent.
"""
runtime.start()  
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: Hi there!
LLMAgent responded: Hello! How can I assist you today?
>>> Hello! How can I assist you today?
>>> This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent received message: This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagree.
LLMAgent responded: I appreciate your feedback! How would you like me to respond?


In [11]:
await runtime.stop()
await runtime.close()

### OK now let's show this at work - let's have 3 agents interact!

Rock paper scissors between 3 agents

In [ ]:
from autogen_ext.models.ollama import OllamaChatCompletionClient


class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent): #same player1 except this one uses ollama 
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [ ]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
        message = Message(content=instruction)
        inner_1 = AgentId("player1", "default") #looks up default id for "player1"
        inner_2 = AgentId("player2", "default") #looks up default id for "player2"
        response1 = await self.send_message(message, inner_1) #send the message to player1 default id
        response2 = await self.send_message(message, inner_2) #send the message to player2 default id
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)


In [ ]:
#establish runtime and register all players & start runtime
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [15]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock
Player 2: rock
In a game of rock, paper, scissors, if both players choose rock, the result is a tie. There is no winner in this round. 

TERMINATE


In [16]:
await runtime.stop()
await runtime.close()